# Political Analysis: Clustering Senators relative to their votes

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sys
from io import StringIO
import time
import math

In [2]:
# What Congress session do you want to scrape?
year = 2017

In [3]:
congress_num = math.floor((year-1787)/2)
session = (year+1)%2 + 1

In [4]:
# Check out list of all Voting Records

base_url = requests.get("https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_"
                        +str(congress_num)
                        +"_"
                        +str(session)
                        +".htm")

print(base_url.status_code)
soup = BeautifulSoup(base_url.content, 'html.parser')

nb_votes_1 = soup.find(id = "secondary_col2")
nb_votes_2 = nb_votes_1.find('table')
nb_votes_3 = nb_votes_2.findAll('td')
nb_votes_4 = nb_votes_3[0].get_text()
nb_votes = int(nb_votes_4.split('\xa0', 1)[0])
print(nb_votes)

200
325


In [5]:
# Scrape the vote descriptions

columns = ['Vote', 'Result', 'Question', 'Issue', 'Date']
vote_desc = pd.DataFrame("", index=np.arange(nb_votes), columns=columns)

vote_desc_table = soup.findAll(class_='contenttext')
vote_desc_table

for i in range(1, nb_votes+1):
    vote_desc['Vote'][nb_votes-i]=vote_desc_table[5*i].get_text()
    vote_desc['Result'][nb_votes-i]=vote_desc_table[5*i+1].get_text()
    vote_desc['Question'][nb_votes-i]=vote_desc_table[5*i+2].get_text()
    vote_desc['Issue'][nb_votes-i]=vote_desc_table[5*i+3].get_text()
    vote_desc['Date'][nb_votes-i]=vote_desc_table[5*i+4].get_text()

vote_desc.to_csv(
    path_or_buf="../input/Vote_Description_"+str(congress_num)+"_"+str(session)+".csv"
    , sep=','
    , index = False
)

In [8]:
# Scrape each vote

for i in range(1,nb_votes+1):
    j = 0
    print(i)

    while j == 0:

        vote_num_for_url = "0"*(5-len(str(i)))+str(i)

        url = requests.get(
            "https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress="
            +str(congress_num)
            +"&session="
            +str(session)
            +"&vote="
            +vote_num_for_url
        )
        soup = BeautifulSoup(url.content
                             , 'html.parser')

        if soup.findAll(class_='newspaperDisplay_3column') == []:
            j = 0 
        else:
            j = 1

    vote_results = soup.findAll(class_='newspaperDisplay_3column')[0].findAll(class_='contenttext')[0].get_text()
    vote_results = vote_results.replace(", Giving Live Pair", "")
    vote_results_csv = StringIO(vote_results)
    vote_results = pd.read_csv(vote_results_csv, sep=",", header=None)
    vote_results.columns = ['0', vote_num_for_url]


    if i == 1:
        all_votes = vote_results
        all_votes.columns = ['Senator', vote_num_for_url]
    else:
        all_votes = all_votes.merge(vote_results, 
                                    left_on='Senator',
                                    right_on='0', 
                                    how='outer')
        for i in range(0, len(all_votes)):
            if pd.isnull(all_votes['Senator'][i]):
                all_votes['Senator'][i]=all_votes['0'][i]
        del all_votes['0']

 <span style="color:red">__If code is stuck on a vote, open the vote's page on your internet browser.__</span>

In [40]:
# Replace Yea = 1 and Nay = -1 (other = 0)

for i in range(1, nb_votes+1):
    vote_num_for_url = "0"*(5-len(str(i)))+str(i)
    all_votes['Vote_'+vote_num_for_url]=0
    all_votes['Vote_'+vote_num_for_url][all_votes[vote_num_for_url]==' Yea']=1
    all_votes['Vote_'+vote_num_for_url][all_votes[vote_num_for_url]==' Nay']=-1
    del all_votes[vote_num_for_url]

/Users/nj995/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/nj995/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [41]:
all_votes['Senator_Name'] = all_votes.Senator.str[:-7]
all_votes['Senator_Party'] = all_votes.Senator.str[-5:-4]
all_votes['Senator_State'] = all_votes.Senator.str[-3:-1]
del all_votes['Senator']

In [42]:
columns = ['Senator_Name', 'Senator_Party', 'Senator_State']
for i in range(1, nb_votes+1):
    vote_num_for_url = "0"*(5-len(str(i)))+str(i)
    columns.append('Vote_'+vote_num_for_url)
all_votes = all_votes[columns]

In [43]:
all_votes.to_csv(
    path_or_buf="../input/Senate_Votes_"+str(congress_num)+"_"+str(session)+".csv"
    , sep=','
    , index = False
)